## Xview Dataset clipping
This is a component designed to clip the dataset provided by xview. 

In [98]:
!pip install Pillow

In [96]:

import os
from PIL import Image


In [90]:

#directory_path is the path to the folder with the unzipped .tif images from xview dataset 
directory_path = os.environ.get("directory_path")

#Each image is cropped using a square window with edge_length window_size which has to be given in number of pixels 
window_size = os.environ.get("window_size")

#step_size is the length in pixels the sliding window is moved after each step
#For tumbling window step_size must equal window_size
step_size = os.environ.get("step_size")


In [91]:

def clear_directory(directory):
    for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            # Check if the item is a file
            if os.path.isfile(item_path):
                # If it's a file, remove it
                os.remove(item_path)
            # If it's a directory, recursively clear it
            elif os.path.isdir(item_path):
                clear_directory(item_path)
                os.rmdir(item_path)
                

In [92]:

def sliding_window(directory_path, window_size, step_size):
    
    parent_dir = os.path.dirname(directory_path)

    clipped = os.path.join(parent_dir, "clipped")
    if os.path.isdir(clipped):
        clear_directory(clipped)
    else: 
        os.makedirs(clipped)
        
    
    for item in os.listdir(directory_path):
        if item == ".DS_Store":
            continue
        item_path = os.path.join(directory_path, item)
        clipped_item = os.path.join(clipped, item)
        os.makedirs(clipped_item)
        
        image = Image.open(item_path)
        width, height = image.size
        
        x_range = [0]
        while(x_range[-1] + step_size + window_size < width):
            x_range += [x_range[-1] + step_size]
        
        y_range = [0]
        while(y_range[-1] + step_size + window_size < height):
            y_range += [y_range[-1] + step_size]
        
        counter = 0
        for x in x_range:
            for y in y_range:
                cropped = image.crop((x,y, x+window_size, y+window_size))

                file_name = "{}".format(y)
                file_name += "_{}".format(x) + ".tif"
                path = os.path.join(clipped_item, file_name)

                cropped.save(path)


In [93]:
sliding_window(directory_path, window_size, step_size)

TypeError: expected str, bytes or os.PathLike object, not NoneType